In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np

### Create a Summary Dataframe (tdf) for Data from the most recent year:

In [48]:
year = 2020
filename = 'PHMSA_Distribution_Data_2010_2020/annual_gas_distribution_' + str(year) + '.xlsx'
tdf = pd.read_excel(filename)
tdf.columns = tdf.iloc[1]
tdf = tdf.drop([0,1]).reset_index(drop=True)
tdf['SRVCS_MILES_TOTAL'] = [tdf.at[i,'AVERAGE_LENGTH']*tdf.at[i,'NUM_SRVCS_TOTAL']/5280 for i in tdf.index]
basic_info_cols = ['OPERATOR_NAME','OPERATOR_ID','OPERATOR_TYPE','STOP','MMILES_TOTAL','SRVCS_MILES_TOTAL','AVERAGE_LENGTH','NUM_SRVCS_TOTAL']
mmiles_historic_cols = [str(i) for i in tdf.columns if "MMILES_BY" in i]
srvs_historic_cols = [str(i) for i in tdf.columns if "NUM_SRVS_BY" in i]
leaks_cols = [str(i) for i in tdf.columns if "TOTAL_LEAKS" in i]
hazleaks_cols = [str(i) for i in tdf.columns if "TOTAL_HAZLEAKS" in i]
tdf = tdf.filter(basic_info_cols + mmiles_historic_cols + srvs_historic_cols + leaks_cols + hazleaks_cols)
tot_leaks = [0 for _ in tdf.index]
tot_hazleaks = [0 for _ in tdf.index]
for i in tdf.index:
    tot_leaks[i] = sum(tdf.iloc[i,31:47])
    tot_hazleaks[i] = sum(tdf.iloc[i,47:])
tdf = tdf.drop(columns=[i for i in tdf.columns if ('TOTAL_LEAKS' in i) or ('TOTAL_HAZLEAKS'in i)])
tdf = tdf.drop(columns=['MMILES_BY_DCD_TOTAL','NUM_SRVS_BY_DCD_TOTAL'])
tdf['TOTAL_LEAKS_'+str(year)] = tot_leaks
tdf['HAZARDOUS_LEAKS_'+str(year)] = tot_hazleaks
tdf['Main Lines over 50 Years Old'] = [int(tdf.loc[i,'MMILES_BY_DCD_1940_TO_1949']+tdf.loc[i,'MMILES_BY_DCD_1940_TO_1949']+tdf.loc[i,'MMILES_BY_DCD_1950_TO_1959']+tdf.loc[i,'MMILES_BY_DCD_1960_TO_1969']+(1.5/10)*tdf.loc[i,'MMILES_BY_DCD_1970_TO_1979']) for i in tdf.index]

In [49]:
tdf.head(2)

1,OPERATOR_NAME,OPERATOR_ID,OPERATOR_TYPE,STOP,MMILES_TOTAL,SRVCS_MILES_TOTAL,AVERAGE_LENGTH,NUM_SRVCS_TOTAL,MMILES_BY_DCD_UNK,MMILES_BY_DCD_PRE1940,MMILES_BY_DCD_1940_TO_1949,MMILES_BY_DCD_1950_TO_1959,MMILES_BY_DCD_1960_TO_1969,MMILES_BY_DCD_1970_TO_1979,MMILES_BY_DCD_1980_TO_1989,MMILES_BY_DCD_1990_TO_1999,MMILES_BY_DCD_2000_TO_2009,MMILES_BY_DCD_2010_TO_2019,MMILES_BY_DCD_2020_TO_2029,NUM_SRVS_BY_DCD_UNK,NUM_SRVS_BY_DCD_PRE1940,NUM_SRVS_BY_DCD_1940_TO_1949,NUM_SRVS_BY_DCD_1950_TO_1959,NUM_SRVS_BY_DCD_1960_TO_1969,NUM_SRVS_BY_DCD_1970_TO_1979,NUM_SRVS_BY_DCD_1980_TO_1989,NUM_SRVS_BY_DCD_1990_TO_1999,NUM_SRVS_BY_DCD_2000_TO_2009,NUM_SRVS_BY_DCD_2010_TO_2019,NUM_SRVS_BY_DCD_2020_TO_2029,TOTAL_LEAKS_2020,HAZARDOUS_LEAKS_2020,Main Lines over 50 Years Old
0,"ABBYVILLE, CITY OF",18,Municipal Owned,KS,3,0.344697,35,52,0,0,0,3,0,0,0,0,0,0,0,0,0,0,51,0,0,0,0,1,0,0,52,0,3
1,ABITA SPRINGS NAT GAS & WATER,27,Municipal Owned,LA,28,5.375000,30,946,0,0,0,0,0,0,0,0,28,0,0,0,0,0,0,0,0,0,0,852,66,28,950,2,0


### Create a Main Mileage and System Age Summary File:

In [15]:
mmiles = tdf.copy()
mmiles = mmiles.filter(['OPERATOR_NAME','OPERATOR_ID','OPERATOR_TYPE','STOP','MMILES_TOTAL','MMILES_BY_DCD_UNK', 'MMILES_BY_DCD_PRE1940','MMILES_BY_DCD_1940_TO_1949', 'MMILES_BY_DCD_1950_TO_1959',
'MMILES_BY_DCD_1960_TO_1969', 'MMILES_BY_DCD_1970_TO_1979','MMILES_BY_DCD_1980_TO_1989', 'MMILES_BY_DCD_1990_TO_1999',
       'MMILES_BY_DCD_2000_TO_2009', 'MMILES_BY_DCD_2010_TO_2019','MMILES_BY_DCD_2020_TO_2029','Main Lines over 50 Years Old'])
mmiles.columns = ['Operator Name','Operator ID','Operator Type','State','Total Main Miles','Main Miles Built in Unknown Decade','Main Miles Built Pre-1940','Main Miles Built 1940-49','Main Miles Built 1950-59','Main Miles Built 1960-69','Main Miles Built 1970-79','Main Miles Built 1980-89','Main Miles Built 1990-99','Main Miles Built 2000-09','Main Miles Built 2010-19','Main Miles Built 2020-29','Main Miles over 50 Years Old']
ages = [1925,1945,1955,1965,1975,1985,1995,2005,2015,2020]
cols = list(mmiles.columns[6:-1])
mileage_age = [0 for _ in mmiles.index]
for i in mmiles.index:
    age = 0
    for j in range(len(ages)):
        age += mmiles.at[i,cols[j]]*(2021-ages[j])
    try:
        mileage_age[i] = int(age/(mmiles.at[i,'Total Main Miles']-mmiles.at[i,'Main Miles Built in Unknown Decade']))
    except:
        None
mmiles['Main Mileage Average Age'] = mileage_age
mmiles = mmiles.filter(['Operator Name','Operator ID','Operator Type','State','Total Main Miles','Main Mileage Average Age','Main Miles over 50 Years Old','Main Miles Built in Unknown Decade','Main Miles Built Pre-1940','Main Miles Built 1940-49','Main Miles Built 1950-59','Main Miles Built 1960-69','Main Miles Built 1970-79','Main Miles Built 1980-89','Main Miles Built 1990-99','Main Miles Built 2000-09','Main Miles Built 2010-19','Main Miles Built 2020-29'])
#mmiles.to_csv('PHMSA_Cleaned_Data/PHMSA_Main_Mileage_2020.csv')


### Create a Hazardous Leaks Time Series File:

In [79]:
hazleaks_cols = [str(i) for i in tdf.columns if "TOTAL_HAZLEAKS" in i]

opid_to_years = dict()
for i in tdf.index:
    opid = tdf.at[i,'OPERATOR_ID']
    opid_to_years[opid] = dict()
    opid_to_years[opid]['Operator Name'] = tdf.at[i,'OPERATOR_NAME']
    opid_to_years[opid]['Operator Type'] = tdf.at[i,'OPERATOR_TYPE']
    opid_to_years[opid]['State'] = tdf.at[i,'STOP']
    for y in range(2010,2021):
        opid_to_years[opid][str(y)] = 0
for i in range(2010,2021):
    year = str(i)
    filename = 'PHMSA_Distribution_Data_2010_2020/annual_gas_distribution_' + str(year) + '.xlsx'
    tdf = pd.read_excel(filename)
    tdf.columns = tdf.iloc[1]
    tdf = tdf.drop([0,1]).reset_index(drop=True)
    for j in tdf.index:
        try:
            opid = tdf.at[j,'OPERATOR_ID']
            opid_to_years[opid][year] = sum(tdf.loc[j,hazleaks_cols])
        except:
            print(opid, i)
hazleaks10 = pd.DataFrame(opid_to_years).T
hazleaks10a = hazleaks10.copy().reset_index()
hazleaks10a.columns = ['Operator ID'] + list(hazleaks10.columns[0:])

144 2010
414 2010
473 2010
473 2010
473 2010
630 2010
705 2010
837 2010
882 2010
887 2010
927 2010
985 2010
1121 2010
1209 2010
1240 2010
1365 2010
1512 2010
1712 2010
1848 2010
2066 2010
2332 2010
2425 2010
2537 2010
2672 2010
2682 2010
2868 2010
3080 2010
3601 2010
4100 2010
4150 2010
4180 2010
4510 2010
4510 2010
4510 2010
4547 2010
4550 2010
4908 2010
4908 2010
4908 2010
5240 2010
5405 2010
5620 2010
6400 2010
6420 2010
6601 2010
8290 2010
10030 2010
10030 2010
10030 2010
10320 2010
11176 2010
11360 2010
11565 2010
11849 2010
11856 2010
11888 2010
12078 2010
12122 2010
12169 2010
12288 2010
12324 2010
12500 2010
12516 2010
12618 2010
13707 2010
13720 2010
13750 2010
13750 2010
13750 2010
13750 2010
13750 2010
13750 2010
13750 2010
13750 2010
13750 2010
13850 2010
13850 2010
14035 2010
14140 2010
14341 2010
15091 2010
15154 2010
15259 2010
15623 2010
17325 2010
17430 2010
17681 2010
18184 2010
18188 2010
18465 2010
18520 2010
18568 2010
18711 2010
18779 2010
19020 2010
19090 2010
19

81 2016
837 2016
887 2016
1209 2016
1512 2016
2066 2016
2616 2016
2616 2016
2616 2016
2616 2016
2616 2016
2616 2016
2616 2016
2620 2016
2672 2016
2868 2016
3601 2016
4180 2016
4547 2016
4550 2016
6400 2016
6420 2016
6601 2016
8290 2016
11176 2016
11360 2016
11856 2016
11888 2016
12078 2016
12288 2016
12618 2016
13131 2016
13131 2016
14035 2016
14341 2016
15154 2016
15259 2016
17040 2016
18465 2016
18520 2016
18568 2016
19340 2016
19670 2016
22840 2016
26085 2016
30529 2016
30575 2016
30639 2016
30914 2016
31260 2016
31366 2016
31467 2016
31467 2016
31668 2016
32069 2016
32123 2016
32190 2016
32241 2016
32261 2016
32382 2016
32388 2016
32399 2016
32521 2016
32585 2016
36417 2016
36672 2016
38881 2016
38910 2016
38997 2016
39103 2016
39143 2016
39254 2016
39319 2016
39457 2016
39466 2016
39503 2016
39535 2016
99128 2016
81 2017
887 2017
1209 2017
1512 2017
2066 2017
2616 2017
2616 2017
2616 2017
2616 2017
2616 2017
2616 2017
2616 2017
2620 2017
2672 2017
2868 2017
3601 2017
4180 2017
454

In [56]:
hazleaks10a.sort_values(by=['2020'],ascending=False)

,Operator ID,Operator Name,Operator Type,State,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,18,"ABBYVILLE, CITY OF",Municipal Owned,KS,0,0,0,0,0,0,0,0,0,0,0
901,18594,"SPUR, CITY OF",Municipal Owned,TX,0,0,0,0,0,0,0,0,0,0,0
899,18584,"SPRINGFIELD, CITY UTILITIES OF",Municipal Owned,MO,0,0,0,0,0,0,0,0,0,0,0
898,18578,SPERRY UTILITY SERVICES AUTH,Municipal Owned,OK,0,0,0,0,0,0,0,0,0,0,0
897,18572,"SPEARVILLE GAS SYSTEM, CITY OF",Municipal Owned,KS,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,7240,HERMANN CITY NATURAL GAS SYSTEM,Municipal Owned,MO,0,0,0,0,0,0,0,0,0,0,0
445,7225,HENNING GAS DEPT,Municipal Owned,TN,0,0,0,0,0,0,0,0,0,0,0
444,7220,HENDERSON MUNICIPAL GAS,Municipal Owned,KY,0,0,0,0,0,0,0,0,0,0,0
443,7215,"HENDERSON GAS DEPT, CITY OF",Municipal Owned,TN,0,0,0,0,0,0,0,0,0,0,0


In [52]:
hazleaks10a.head(40)

,Operator ID,Operator Name,Operator Type,State,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,18,"ABBYVILLE, CITY OF",Municipal Owned,KS,0,0,0,0,0,0,0,0,0,0,0
1,27,ABITA SPRINGS NAT GAS & WATER,Municipal Owned,LA,0,0,0,0,0,0,0,0,0,0,0
2,45,"ADAIRSVILLE, CITY OF",Municipal Owned,GA,0,0,0,0,0,0,0,0,0,0,0
3,49,TOWN OF ADAMSVILLE GAS DEPT,Municipal Owned,TN,0,0,0,0,0,0,0,0,0,0,0
4,54,"ADEL GAS DEPT, CITY OF",Municipal Owned,GA,0,0,0,0,0,0,0,0,0,0,0
5,180,SPIRE ALABAMA INC.,Investor Owned,AL,0,0,0,0,0,0,0,0,0,0,0
6,207,ALASKA PIPELINE CO,Privately Owned,AK,0,0,0,0,0,0,0,0,0,0,0
7,225,ALBANY MUNICIPAL GAS CO,Municipal Owned,MO,0,0,0,0,0,0,0,0,0,0,0
8,234,ALBANY WATER GAS & LIGHT COMMISSION,Municipal Owned,GA,0,0,0,0,0,0,0,0,0,0,0
9,252,"ALEDO GAS DEPT, CITY OF",Municipal Owned,IL,0,0,0,0,0,0,0,0,0,0,0
